In [63]:
#api key : beec8fe6e17edd7b79a40552e38db419
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
import warnings
import numpy as np


api_key = "beec8fe6e17edd7b79a40552e38db419"

def gen_latlon(city):
    t_start=1610107200
    t_end=1610107200
    url = "http://api.openweathermap.org/geo/1.0/direct?q=%s &limit=5&appid=%s" % (city, api_key)
    response  = requests.get(url)
    data = json.loads(response.text)
    lat = data[0]["lat"]
    lon = data[0]["lon"]
    print(lat)
    print(lon)
    df = gen_point_data(lat, lon, t_start, t_end)
    return df

    
def gen_point_data(lat, lon, t_start, t_end):
    url2 = "http://api.openweathermap.org/data/2.5/air_pollution/history?lat=%s&lon=%s&start=1610107200&end=1610107200&appid=%s" % (lat, lon, api_key)
    response = requests.get(url2)
    data2 = json.loads(response.text)
    df = pd.json_normalize(data2["list"])
    print(df)
    return df
    
city = ["New York", 
"Los Angeles", 
"Chicago",
"Miami", 
"Dallas", 
"Philadelphia", 
"Houston", 
"Atlanta", 
"Washington", 
"Boston", 
"Phoenix", 
"Seattle", 
"San Francisco", 
"Detroit", 
"San Diego", 
"Minneapolis", 
"Tampa", 
"Denver", 
"Brooklyn"]
df = gen_latlon(city[0])
df.to_csv('data.csv', index=False)



40.7143
-74.006
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         1  ...            10.36            3.04

[1 rows x 10 columns]


In [64]:
# for c in city[1:]:
#     df = gen_latlon(c)
#     df.to_csv('data.csv', dtype={
#         'string_col': 'float16',
#         'int_col': 'float16'
#     } ,index=False, mode='a', header=False)
# dfz = pd.read_csv('data.csv')
# dfz.head()
nI = len(city)
for c in range(1, nI):
    print(city[c])
    df = gen_latlon(city[c])
    df.to_csv('data.csv', index=False, mode='a', header=False)
dfz = pd.read_csv('data.csv')
dfz.head()

Los Angeles
34.0522
-118.2437
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         4  ...            57.47            7.35

[1 rows x 10 columns]
Chicago
41.85
-87.65
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         1  ...             1.94            0.28

[1 rows x 10 columns]
Miami
25.7743
-80.1937
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         2  ...             13.4            0.05

[1 rows x 10 columns]
Dallas
32.7831
-96.8067
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         1  ...              4.9            0.36

[1 rows x 10 columns]
Philadelphia
39.9523
-75.1638
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         2  ...            12.57            3.17

[1 rows x 10 columns]
Houston
29.7633
-95.3633
           dt  main.aqi  ...  components.pm10  components.nh3
0  1610107200         1  ...              4.4     

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3
0,1610107200,1,370.50,1.48,38.04,17.88,7.51,7.27,10.36,3.04
1,1610107200,4,894.55,78.68,78.83,0.00,13.83,40.59,57.47,7.35
2,1610107200,1,253.68,0.00,6.68,60.80,3.34,1.29,1.94,0.28
3,1610107200,2,263.69,0.00,13.88,47.92,4.65,11.37,13.40,0.05
4,1610107200,1,257.02,0.00,11.48,47.92,2.83,4.18,4.90,0.36


In [65]:
#round and convert to int
dfz.round(0).astype(int)

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3
0,1610107200,1,370,1,38,18,8,7,10,3
1,1610107200,4,895,79,79,0,14,41,57,7
2,1610107200,1,254,0,7,61,3,1,2,0
3,1610107200,2,264,0,14,48,5,11,13,0
4,1610107200,1,257,0,11,48,3,4,5,0
5,1610107200,2,387,1,42,17,8,10,13,3
6,1610107200,1,250,0,11,56,2,3,4,1
7,1610107200,1,297,0,15,44,3,8,9,0
8,1610107200,1,270,0,4,46,0,5,5,0
9,1610107200,1,264,0,13,48,3,2,3,1


In [66]:
# #normalize
from sklearn import preprocessing

x = dfz.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dfz = pd.DataFrame(x_scaled)
dfz.head()



,0,1,2,3,4,5,6,7,8,9
0,0.0,0.000000,0.218897,0.01881,0.457508,0.219279,0.422741,0.158130,0.158073,0.374074
1,0.0,1.000000,1.000000,1.00000,1.000000,0.000000,0.791254,0.941024,0.954522,0.906173
2,0.0,0.000000,0.044775,0.00000,0.040431,0.745646,0.179592,0.017622,0.015723,0.033333
3,0.0,0.333333,0.059695,0.00000,0.136188,0.587687,0.255977,0.254464,0.209467,0.004938
4,0.0,0.000000,0.049753,0.00000,0.104269,0.587687,0.149854,0.085526,0.065765,0.043210


In [67]:
dfz.shape[0]

19

In [69]:
# Split traing and test
train = dfz[:12]
test = dfz[12:]

In [70]:
pca = PCA(whiten=True)
pca.fit(dfz)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([0.8117772 , 0.91064993, 0.95248322, 0.97293425, 0.99021732,
       0.99750681, 0.99860433, 0.99947538, 1.        , 1.        ])

In [79]:
pca = PCA(n_components=2,whiten=True)
pca = pca.fit(dfz)
dataPCA = pca.transform(dfz)

